In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

/Users/seyeong/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
data = np.load('datasets/65_MachineryFault.npz', allow_pickle=True)
X, y = data['X'], data['y']

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# 데이터셋 분할 (학습 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [7]:
# LSTM 모델 정의
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



In [8]:
# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
# 모델 학습
model.fit(X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), y_train, epochs=10, batch_size=32, validation_data=(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)), y_test))


Epoch 1/10
56250/56250 [==============================] - 172s 3ms/step - loss: 0.1570 - accuracy: 0.9372 - val_loss: 0.1222 - val_accuracy: 0.9517
Epoch 2/10
56250/56250 [==============================] - 161s 3ms/step - loss: 0.1238 - accuracy: 0.9510 - val_loss: 0.1131 - val_accuracy: 0.9549
Epoch 3/10
56250/56250 [==============================] - 160s 3ms/step - loss: 0.1178 - accuracy: 0.9533 - val_loss: 0.1125 - val_accuracy: 0.9555
Epoch 4/10
56250/56250 [==============================] - 166s 3ms/step - loss: 0.1146 - accuracy: 0.9546 - val_loss: 0.1088 - val_accuracy: 0.9570
Epoch 5/10
56250/56250 [==============================] - 201s 4ms/step - loss: 0.1125 - accuracy: 0.9554 - val_loss: 0.1078 - val_accuracy: 0.9574
Epoch 6/10
56250/56250 [==============================] - 186s 3ms/step - loss: 0.1107 - accuracy: 0.9560 - val_loss: 0.1056 - val_accuracy: 0.9584
Epoch 7/10
56250/56250 [==============================] - 184s 3ms/step - loss: 0.1093 - accuracy: 0.9568 - val_

In [11]:
# 예측
y_pred = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))


14063/14063 [==============================] - 13s 880us/step


In [13]:
# 결과 출력
print(y_pred)   

[[0.01705359]
 [0.9999998 ]
 [0.99999994]
 ...
 [0.00313937]
 [0.99989545]
 [0.9999999 ]]


In [18]:
# 이진 클래스로 다시 변환
y_pred_prob = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1], 1)))
y_pred = (y_pred_prob > 0.8).astype(int)


14063/14063 [==============================] - 12s 853us/step


In [19]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred.flatten())
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.95
